[Home](../index.ipynb) / ADC-Wandler
***
<span style="font-size:20pt;">ADC-Wandler</span>

---
# Lösungsvorschlag

## Aufgabe
* Mit einem Arduino kompatiblen Board soll eine automatische Dimmerschaltung entwickelt werden die obiges Problem behebt:  
  die Beleuchtung soll bei weniger Umgebungslicht automatisch schwächer werden.
* Die Beleuchtung soll durch eine externe blaue LED realisiert werden, der Umgebungshelligkeitssensor mit einem LDR.  
  Die minimale und die maximale Helligkeit sollen jeweils über eine Variable festgelegt werden (sie sollen später mit einem Rotary-Encoder eingestellt werden können).
  
## Lösung
Mit der angegebenen Beschaltung ergibt sich: je dunkler die Umgebung, desto größer der Wert am ADC, desto dunkler soll die LED leuchten.  
* Von 0 bis zum ADC-Wert `VALUE_MIN` soll die LED mit dem Wert `LED_DUTY_MAX` leuchten
* Vom ADC-Wert `VALUE_MIN` bis zu `VALUE_MAX` soll die LED linear dunkler werden
* Ab dem ADC-Wert `VALUE_MAX` soll die LED mit dem minimalen Wert  `LED_DUTY_MIN` leuchten.

Somit ergibt sich zwischen `VALUE_MIN` und `VALUE_MAX` die Gerade mit der Steigung `VALUE2DUTY_M = (LED_DUTY_MAX-LED_DUTY_MIN)/(VALUE_MAX-VALUE_MIN)` und dem y-Achsenabschnitt `VALUE2DUTY_T = LED_DUTY_MIN - VALUE2DUTY_M * VALUE_MIN`.

|Schaltung|Graph|
|:---:|:---:|
|[<img src="images/LoesungSchaltung.png" width="200">](images/LoesungSchaltung.png) |[<img src="images/LoesungGraph.png" width="300">](images/LoesungGraph.png) |

|HelTec WiWi LoRa 32 V2|HelTec WiWi Kit 32 V2|
|:---:|:---:|
|[<img src="images/AutomaticDimmer_HelTecWiFiLoRa32V2.png" width="400">](images/AutomaticDimmer_HelTecWiFiLoRa32V2.png) |[<img src="images/AutomaticDimmer_HelTecWiFiKit32V2.png" width="400">](images/AutomaticDimmer_HelTecWiFiKit32V2.png) |

In [ ]:
#######################################
# Automatic dimmer
#######################################

%serialconnect
# --port=COM3 # for Windows with more than one COM port

#======================================
# Constants:

PIN_ADC = 32
PIN_LED = 14

#--------------------------------------

FREQUENCY = 1000

VALUE_MIN = 65536/2
VALUE_MAX =     0

LED_DUTY_MIN = 20
LED_DUTY_MAX = FREQUENCY


#--------------------------------------

VALUE2DUTY_M = (LED_DUTY_MAX-LED_DUTY_MIN)/(VALUE_MAX-VALUE_MIN)
VALUE2DUTY_T = LED_DUTY_MIN - VALUE2DUTY_M * VALUE_MIN


#======================================
# init ADC and LED

from machine import ADC, PWM, Pin

#adc = ADC(0)       # Croduino
adc = ADC(Pin(PIN_ADC) ) # HelTec, NodeMCU
adc.atten(ADC.ATTN_6DB)


pinLED = PWM( Pin(PIN_LED, Pin.OUT), freq=FREQUENCY, duty=LED_DUTY_MIN)


#======================================
# Loop "forever"


iDutyLast = LED_DUTY_MIN

try: # try ... except ... finally: for cleaning up
#---------------------------------------------
    
    while True:
        iValue = adc.read_u16()  # read a raw analog value in the range 0-65535 (= 2^16 - 1)
        #iValue = adc.read()      # Croduino: read a raw analog value in the range 0-511 (= 2^7 - 1)
        
        iDuty = VALUE2DUTY_M * iValue + VALUE2DUTY_T
        
        if   iDuty < LED_DUTY_MIN : iDuty = LED_DUTY_MIN
        elif iDuty > LED_DUTY_MAX : iDuty = LED_DUTY_MAX
        
        iDuty = round( iDuty )

        if iDuty != iDutyLast:
            pinLED.duty(iDuty)
            iDutyLast = iDuty

#---------------------------------------------
# If user aborts:            
except KeyboardInterrupt: pass # Skip error messeage
finally:
    pinLED.deinit() # Free PWM-resource

print( "\nDone." )